In [1]:
import numpy as np
import setigen as stg
from blimpy import Waterfall
import matplotlib.pyplot as plt
import random
import os
from astropy import units as u
from tqdm import tqdm
from sklearn.metrics import silhouette_score
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers

os.environ["CUDA_VISIBLE_DEVICES"]="2"


2023-08-24 09:27:55.846510: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
print('GPU name: ', tf.config.experimental.list_physical_devices('GPU'))

GPU name:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
def painting(data):
    all_data = []
    labels = []
    for c in range(num_classes):
        drift = 2*random.random()*(-1)**random.randint(0,2)
        snr = random.randint(100, 150)
        width = random.randint(20, 50)
        for s in range(num_samples_per_class):
            index = random.randint(0, data.shape[0]-1)
            window = data[index, :,:]
            
            start = random.randint(50, 180)
            
            frame = stg.Frame.from_data(df=2.7939677238464355*u.Hz,
                                        dt=18.253611008*u.s,
                                        fch1=1289*u.MHz,
                                        ascending=True,
                                        data=window)
            frame.add_signal(stg.constant_path(
                                        f_start=frame.get_frequency(index=start),
                                       drift_rate=drift*u.Hz/u.s),
                                      stg.constant_t_profile(level=frame.get_intensity(snr=snr)),
                                      stg.gaussian_f_profile(width=width*u.Hz),
                                      stg.constant_bp_profile(level=1))
            all_data.append(frame.data)
            labels.append(c)
    all_data = np.array(all_data)
    labels = np.vstack(labels)
    return all_data, labels

In [4]:
import cv2
import numpy as np



In [5]:
from tqdm import tqdm
import gc
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.beta = 8
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(
            name="kl_loss"
        )
        self.kl_additional = tf.keras.losses.KLDivergence()
    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
        ]
    def gaussanity_loss(self, data, base):
        return self.kl_additional(data, base)
    
    def train_step(self, data_in):
        data = data_in
        print(data.shape)
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.binary_crossentropy(data, reconstruction), axis=(1, 2)
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            
            
            total_loss = reconstruction_loss + self.beta * kl_loss
        
        mse_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.mse(data, reconstruction), axis=(1, 2)
                )
            )
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        self.reconstruction_loss_tracker.update_state(mse_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
        }
    def test_step(self, data_in):
        data, _ = data_in
        z_mean, z_log_var, z = self.encoder(data)
        reconstruction = self.decoder(z)
        reconstruction_loss = tf.reduce_mean(
            tf.reduce_sum(
                keras.losses.binary_crossentropy(data, reconstruction), axis=(1, 2)
            )
        )
        kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))

        total_loss = reconstruction_loss + self.beta * kl_loss 
        
        mse_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.mse(data, reconstruction), axis=(1, 2)
                )
            )
        self.total_loss_tracker.update_state(total_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        self.reconstruction_loss_tracker.update_state(mse_loss)
        return {
            "test_loss": self.total_loss_tracker.result(),
            "test_kl_loss": self.kl_loss_tracker.result(),
            "test_reconstruction_loss": self.reconstruction_loss_tracker.result()
        }
    def __call__ (self, inputs):
        return self.decoder(self.encoder(inputs)[0])

In [6]:
class Sampling(layers.Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [7]:
latent_dim = 10
time_samples = 16
freq_sample =  256
encoder_inputs = keras.Input(shape=(time_samples, freq_sample, 1))
x = layers.Conv2D(32, 3, activation="relu", strides=1, padding="same")(encoder_inputs)
x = layers.MaxPool2D(pool_size=(1, 2))(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(64, 3, activation="relu", strides=1, padding="same")(x)
x = layers.MaxPool2D(pool_size=(1, 2))(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(64, 3, activation="relu", strides=1, padding="same")(x)
x = layers.MaxPool2D(pool_size=(1, 2))(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(128, 3, activation="relu", strides=1, padding="same")(x)
x = layers.MaxPool2D(pool_size=(1, 2))(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(128, 3, activation="relu", strides=1, padding="same")(x)
x = layers.MaxPool2D(pool_size=(1, 2))(x)
x = layers.BatchNormalization()(x)
x_shape = x.shape
x = layers.Flatten()(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.BatchNormalization()(x)
x = layers.Dense(32, activation="relu")(x)
x = layers.BatchNormalization()(x)


z_mean = layers.Dense(32, activation="relu")(x)
z_mean = layers.BatchNormalization()(z_mean)
z_mean = layers.Dense(latent_dim, name="z_mean")(z_mean)

z_log_var = layers.Dense(32, activation="relu")(x)
z_log_var = layers.BatchNormalization()(z_log_var)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(z_log_var)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 16, 256, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 conv2d (Conv2D)                (None, 16, 256, 32)  320         ['input_1[0][0]']                
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 16, 128, 32)  0           ['conv2d[0][0]']                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 16, 128, 32)  128        ['max_pooling2d[0][0]']    

2023-08-24 09:27:57.807058: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-24 09:27:58.194130: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13888 MB memory:  -> device: 0, name: NVIDIA RTX A4000, pci bus id: 0000:a1:00.0, compute capability: 8.6


Total params: 4,485,812
Trainable params: 4,484,276
Non-trainable params: 1,536
__________________________________________________________________________________________________


In [8]:
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(256, activation="relu")(latent_inputs)
x = layers.BatchNormalization()(x)
x = layers.Dense(x_shape[1]* x_shape[2]* x_shape[3], activation="relu")(x)
x = layers.BatchNormalization()(x)
x = layers.Reshape((x_shape[1], x_shape[2], x_shape[3]))(x)
x = layers.Conv2DTranspose(128, 3, activation="relu", strides=2, padding="same")(x)
x = layers.MaxPool2D(pool_size=(2, 1))(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2DTranspose(128, 3, activation="relu", strides=2, padding="same")(x)
x = layers.MaxPool2D(pool_size=(2, 1))(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.MaxPool2D(pool_size=(2, 1))(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.MaxPool2D(pool_size=(2, 1))(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
x = layers.MaxPool2D(pool_size=(2, 1))(x)
x = layers.BatchNormalization()(x)
decoder_outputs = layers.Conv2DTranspose(1, 3, activation="linear", padding="same")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 10)]              0         
                                                                 
 dense_4 (Dense)             (None, 256)               2816      
                                                                 
 batch_normalization_9 (Batc  (None, 256)              1024      
 hNormalization)                                                 
                                                                 
 dense_5 (Dense)             (None, 16384)             4210688   
                                                                 
 batch_normalization_10 (Bat  (None, 16384)            65536     
 chNormalization)                                                
                                                                 
 reshape (Reshape)           (None, 16, 8, 128)        0   

In [9]:
autoencoder = VAE(encoder, decoder)
autoencoder.compile(optimizer=keras.optimizers.Adam(learning_rate = 1e-3))
# autoencoder.load_weights("../b-vae/models/full-weights-"+'07-02-2023-15-19-23')
autoencoder.load_weights("../b-vae/models/full-weights-"+'08-23-2023-21-05-49')


In [10]:
def normalize(data):
    epsilon = 1
    min_val = data.min()
    data = data - min_val + epsilon
    new_data = np.log(data)
    min_val = data.min()
    max_val = data.max()
    final_data = (data - min_val) / (max_val - min_val)
    return final_data
    
def normalize_data(data):
    for i in tqdm(range(data.shape[0])):
        data[i,:,:] = normalize(data[i,:,:])
    return data

In [11]:
def idealized(num=10_000):
    drift = drift = (-1)**(random.randint(0,2)) * 2 *random.random()
    snr = 50*random.random() +20
    width = 50*random.random() +20
    start = random.randint(50, 180)
    data = []
    labels = []
    for tag in range(4):
        label_vec = np.zeros(4)
        label_vec[tag] = 1
        for i in range(num):
            if tag == 0:
#             everything but drif
                snr = 50*random.random() +20
                width = 50*random.random() +20
                start = random.randint(50, 180)
            elif tag == 1:
#             everything but snr
                drift = (-1)**(random.randint(0,2)) * 2 *random.random()
                width = 50*random.random() +20
                start = random.randint(50, 180)
            elif tag == 2:
#             everything but width
                drift = (-1)**(random.randint(0,2)) * 2 *random.random()
                snr = 50*random.random() +20
                start = random.randint(50, 180)
            elif tag == 3:
#             everything but start
                drift = (-1)**(random.randint(0,2)) * 2 *random.random()
                snr = 50*random.random() +20
                width = 50*random.random() +20

            frame = stg.Frame(fchans=256*u.pixel,
                              tchans=16*u.pixel,
                              df=2.7939677238464355*u.Hz,
                              dt=18.253611008*u.s,
                              fch1=6095.214842353016*u.MHz)
            noise = frame.add_noise(x_mean=1, noise_type='chi2')
            frame.add_signal(stg.constant_path(
                                        f_start=frame.get_frequency(index=start),
                                       drift_rate=drift*u.Hz/u.s),
                                      stg.constant_t_profile(level=frame.get_intensity(snr=snr)),
                                      stg.gaussian_f_profile(width=width*u.Hz),
                                      stg.constant_bp_profile(level=1))
            data.append(frame.data)
            labels.append(label_vec)
    data = np.array(data)
    labels = np.vstack(labels)
    return data, labels

In [12]:
def absolute_diff(X, labels):
    new_x = []
    new_labels = []
    one = np.arange(0,10000)
    two = np.arange(10000,20000)
    three = np.arange(20000,30000)
    four = np.arange(30000,40000)
    for i in range(X.shape[0] -1):
        if np.argmax(labels[i,:]) == np.argmax(labels[i+1,:]): 
            index = np.argmax(labels[i,:])
            if index == 0:
                sample = np.random.choice(one, size = 1000)
            elif index == 1:
                sample = np.random.choice(two, size = 1000)
            elif index == 2:
                sample = np.random.choice(three, size = 1000)
            elif index == 3:
                sample = np.random.choice(four, size = 1000)
            diff = abs(X[i,:] - X[sample,:])
            diff = np.mean(diff, axis = 0)
            new_x.append(diff)
            new_labels.append(labels[i,:])
            
    return np.array(new_x), np.vstack(new_labels)

In [13]:
import os
from tqdm import tqdm
import keras
total_scores = []
for i in tqdm(range(10)):
    idealized_signals, labels = idealized(num = 10_000)
    input_data = np.expand_dims(normalize_data(idealized_signals), axis = -1)
    print(input_data[0,:,:].max(), input_data[0,:,:].min())
    splits = input_data.shape[0]//1_000
    X = []
    print(input_data.shape)
    for i in range(splits):
        tensor = tf.convert_to_tensor(input_data[i*1_000:(i+1)*1_000, :,:,:], dtype=tf.float32)
        X.append(autoencoder.encoder(tensor)[0])
    X = np.vstack(X)
    print(X.shape)
    print(labels.shape)
    X, labels = absolute_diff(X, labels)
    print(X.shape, labels.shape)
    X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.33, random_state=42)

    linear = keras.Sequential(name="my_sequential")
    linear.add(layers.Dense(4, activation="softmax", name="layer2"))

    linear.compile(loss='categorical_crossentropy', 
                   optimizer=keras.optimizers.Adam(learning_rate = 5e-3), metrics=['accuracy'])
    linear.fit(X_train, y_train, epochs=300, batch_size=128, validation_split = 0.1, shuffle=True)
    results = linear.evaluate(X_test, y_test)
    print(results)
    total_scores.append(results[-1])

100%|████████████████████████████████████████████████████████████████████████████| 40000/40000 [00:03<00:00, 10914.39it/s]


1.0 0.0
(40000, 16, 256, 1)


2023-08-24 09:28:37.448003: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8401
2023-08-24 09:28:40.294843: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


(40000, 10)
(40000, 4)
(39996, 10) (39996, 4)
Epoch 1/300
189/189 [==============================] - 1s 2ms/step - loss: 1.1391 - accuracy: 0.5395 - val_loss: 0.9229 - val_accuracy: 0.7701
Epoch 2/300
189/189 [==============================] - 0s 2ms/step - loss: 0.8088 - accuracy: 0.8175 - val_loss: 0.7192 - val_accuracy: 0.8418
Epoch 3/300
189/189 [==============================] - 0s 2ms/step - loss: 0.6563 - accuracy: 0.8637 - val_loss: 0.6098 - val_accuracy: 0.8668
Epoch 4/300
189/189 [==============================] - 0s 2ms/step - loss: 0.5677 - accuracy: 0.8794 - val_loss: 0.5414 - val_accuracy: 0.8746
Epoch 5/300
189/189 [==============================] - 0s 2ms/step - loss: 0.5081 - accuracy: 0.8885 - val_loss: 0.4913 - val_accuracy: 0.8858
Epoch 6/300
189/189 [==============================] - 0s 2ms/step - loss: 0.4647 - accuracy: 0.8940 - val_loss: 0.4551 - val_accuracy: 0.8866
Epoch 7/300
189/189 [==============================] - 0s 2ms/step - loss: 0.4312 - accuracy: 0.

 10%|████████▌                                                                            | 1/10 [02:34<23:10, 154.49s/it]

...

[0.18100179731845856, 0.9316614866256714]



100%|████████████████████████████████████████████████████████████████████████████| 40000/40000 [00:03<00:00, 10912.62it/s]


1.0 0.0
(40000, 16, 256, 1)
(40000, 10)
(40000, 4)
(39996, 10) (39996, 4)
Epoch 1/300
189/189 [==============================] - 1s 2ms/step - loss: 1.3423 - accuracy: 0.4712 - val_loss: 0.9982 - val_accuracy: 0.6817
Epoch 2/300
189/189 [==============================] - 0s 2ms/step - loss: 0.8564 - accuracy: 0.7470 - val_loss: 0.7683 - val_accuracy: 0.7739
Epoch 3/300
189/189 [==============================] - 0s 2ms/step - loss: 0.7019 - accuracy: 0.8001 - val_loss: 0.6649 - val_accuracy: 0.7963
Epoch 4/300
189/189 [==============================] - 0s 2ms/step - loss: 0.6214 - accuracy: 0.8159 - val_loss: 0.6013 - val_accuracy: 0.8119
Epoch 5/300
189/189 [==============================] - 0s 2ms/step - loss: 0.5708 - accuracy: 0.8237 - val_loss: 0.5617 - val_accuracy: 0.8138
Epoch 6/300
189/189 [==============================] - 0s 2ms/step - loss: 0.5371 - accuracy: 0.8246 - val_loss: 0.5307 - val_accuracy: 0.8216
Epoch 7/300
189/189 [==============================] - 0s 2ms/step -

100%|████████████████████████████████████████████████████████████████████████████████████| 10/10 [25:42<00:00, 154.28s/it]

[0.372636616230011, 0.8385483622550964]


In [14]:
print(np.mean(total_scores))
print(np.std(total_scores))

0.8712099373340607
0.04075216568582612
tensorflow      WARNING  Detecting that an object or model or tf.train.Checkpoint is being deleted with unrestored values. See the following logs for the specific values in question. To silence these warnings, use `status.expect_partial()`. See https://www.tensorflow.org/api_docs/python/tf/train/Checkpoint#restorefor details about the status object returned by the restore function.
tensorflow      WARNING  Value in checkpoint could not be found in the restored object: (root).optimizer._variables.1
tensorflow      WARNING  Value in checkpoint could not be found in the restored object: (root).optimizer._variables.2
tensorflow      WARNING  Value in checkpoint could not be found in the restored object: (root).optimizer._variables.3
tensorflow      WARNING  Value in checkpoint could not be found in the restored object: (root).optimizer._variables.4
tensorflow      WARNING  Value in checkpoint could not be found in the restored object: (root).optimizer